# Solvent Small Experiment

Question: Could be use data from minotiry solvents? Does it add value/relevant information?

Descriptions:
- 3 solvent availables, one majority and two minorities
- sample from majority and minorities: 3 samples
- train in majority
- evaluate in 3 samples and compare results, is there any particular improvement in majority?

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from src.config import chemical_inventory_path, raw_data_path
from src.data import notebook_utils as utils
from src.constants import GBL_INCHI_KEY, DMSO_INCHI_KEY, DMF_INCHI_KEY, \
                        INCHI_TO_CHEMNAME, TARGET_COL, RXN_FEAT_NAME, ORGANOAMONIUM_INCHI_KEY_COL
from src import plot_utils

In [45]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
plt.style.reload_library()
import matplotlib.patches as mpatches

## Generate datasets

In [3]:
%cd ../..

/Users/mticona/Documents/tesis/licentiate-thesis-repo


In [46]:
SEED = 105
ALGORITHM = 'rf'

solvents_inchies = [GBL_INCHI_KEY, DMSO_INCHI_KEY, DMF_INCHI_KEY]
solvents = [INCHI_TO_CHEMNAME[inchie] for inchie in solvents_inchies]

plot_solvents = {'Gamma-Butyrolactone': "GBL",
                 'Dimethyl sulfoxide':"DMSO",
                 'Dimethylformamide': "DMF"}

In [47]:
df = utils.read_data(raw_data_path, organic_key=True, solvent=GBL_INCHI_KEY).drop_duplicates()

/Users/mticona/Documents/tesis/licentiate-thesis-repo/src/data/notebook_utils.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,col] = (df[TARGET_COL] == 4).astype(int)


In [13]:
df.shape

(5644, 72)

In [25]:
raw_cols = utils.get_deafult_model_columns()

import json
f = open('data/metadata/map_chem_feat_names.json')
map_names = json.load(f)

df_amines = pd.read_csv("data/metadata/type_var_fq_bins.csv")

chem_feat = [ col for col in raw_cols if map_names.get(col," ") in list(df_amines.propiedad)]

len(chem_feat)

rxn_feat = utils.concentration_feats()

model_cols = chem_feat + rxn_feat + [TARGET_COL]

In [26]:
df = df[model_cols+[ORGANOAMONIUM_INCHI_KEY_COL]]

In [40]:
PATH_DATA = 'data/ensemble/'
FILE_EVAL = 'validation_set.csv'
FILE_TEST_TRAIN = 'train_test_set.csv'

In [31]:
# Validation sample
df_val = df.groupby([TARGET_COL, ORGANOAMONIUM_INCHI_KEY_COL])\
                            .sample(frac=0.1, random_state=SEED)

In [42]:
df_val.to_csv(PATH_DATA+FILE_EVAL)

In [36]:
not_val_idx = [idx for idx in df.index if idx not in df_val.index]

In [38]:
df_test_train = df.loc[not_val_idx, :]

In [39]:
df.shape, df_val.shape, df_test_train.shape

((5644, 62), (567, 62), (5077, 62))

In [43]:
df_test_train.to_csv(PATH_DATA+FILE_TEST_TRAIN)